In [1]:
pip install pandas scikit-learn joblib


In [2]:
import pandas as pd

# Path to your CSV file (adjust if subfolders exist inside "real_recipe")
csv_path = "/content/drive/My Drive/real_recipe/recipes.csv"

df = pd.read_csv(csv_path, low_memory=False)
print(df.head())


   RecipeId                               Name  AuthorId      AuthorName  \
0        38  Low-Fat Berry Blue Frozen Dessert      1533          Dancer   
1        39                            Biryani      1567        elly9812   
2        40                      Best Lemonade      1566  Stephen Little   
3        41     Carina's Tofu-Vegetable Kebabs      1586         Cyclopz   
4        42                       Cabbage Soup      1538       Duckie067   

  CookTime PrepTime TotalTime         DatePublished  \
0    PT24H    PT45M  PT24H45M  1999-08-09T21:46:00Z   
1    PT25M     PT4H   PT4H25M  1999-08-29T13:12:00Z   
2     PT5M    PT30M     PT35M  1999-09-05T19:52:00Z   
3    PT20M    PT24H  PT24H20M  1999-09-03T14:54:00Z   
4    PT30M    PT20M     PT50M  1999-09-19T06:19:00Z   

                                         Description  \
0  Make and share this Low-Fat Berry Blue Frozen ...   
1  Make and share this Biryani recipe from Food.com.   
2  This is from one of my  first Good House 

In [3]:
print(df.columns.tolist())

['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions']


In [4]:
import pandas as pd
import spacy
import os

# Load spaCy NLP model
print("🔁 Loading spaCy NLP model...")
nlp = spacy.load("en_core_web_sm")

# Load your CSV
csv_file = "recipes.csv"

if not os.path.exists(csv_file):
    print(f"❌ File '{csv_file}' not found. Please place your recipes.csv in this folder.")
    exit()

print("📂 Loading recipes dataset (please wait)...")
try:
    df = pd.read_csv(csv_file, low_memory=False)
except Exception as e:
    print("❌ Failed to load the CSV. Check format or encoding.")
    print(str(e))
    exit()

# Normalize column names
df.columns = df.columns.str.lower()

# Check required columns
required = ["name", "recipeingredientparts", "recipeinstructions"]
missing = [col for col in required if col not in df.columns]

if missing:
    print(f"❌ Missing columns in CSV: {missing}")
    exit()

# Function to find recipe
def find_recipe(query):
    doc = nlp(query)

    # Extract noun phrase (food name)
    food_name = None
    for chunk in doc.noun_chunks:
        food_name = chunk.text.strip()
        break

    if not food_name:
        print("❌ Couldn't understand the food name. Try again.")
        return

    print(f"\n🔍 Searching for recipes related to: '{food_name}'...\n")

    # Search in the 'name' column
    results = df[df['name'].str.contains(food_name, case=False, na=False)]

    if results.empty:
        print(f"❌ No recipe found for '{food_name}'. Try something else.")
    else:
        recipe = results.iloc[0]

        title = recipe.get('name', 'N/A')
        ingredients = recipe.get('recipeingredientparts', 'N/A')
        instructions = recipe.get('recipeinstructions', 'N/A')

        # Ensure ingredients are printed nicely
        if isinstance(ingredients, str) and ingredients.startswith("[") and ingredients.endswith("]"):
            try:
                import ast
                ingredients = ast.literal_eval(ingredients)
                ingredients = "\n- " + "\n- ".join(map(str, ingredients))
            except:
                pass

        if isinstance(instructions, str) and instructions.startswith("[") and instructions.endswith("]"):
            try:
                import ast
                instructions = ast.literal_eval(instructions)
                instructions = "\n".join(f"{i+1}. {step}" for i, step in enumerate(instructions))
            except:
                pass

        print(f"🍽️  Recipe: {title}")
        print("\n🧂 Ingredients:\n", ingredients)
        print("\n👨‍🍳 Instructions:\n", instructions)
        print("\n" + "="*50)

# Main Loop
if __name__ == "__main__":
    print("\n👩‍🍳 Welcome to the AI Recipe Generator!")
    print("Type a dish name like 'butter chicken', 'chocolate cake', or 'pasta'.")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("🍲 What recipe do you want? → ")
        if user_input.lower().strip() in ["exit", "quit"]:
            print("👋 Exiting. Happy cooking!")
            break
        find_recipe(user_input)


🔁 Loading spaCy NLP model...
❌ File 'recipes.csv' not found. Please place your recipes.csv in this folder.
📂 Loading recipes dataset (please wait)...
❌ Failed to load the CSV. Check format or encoding.
[Errno 2] No such file or directory: 'recipes.csv'

👩‍🍳 Welcome to the AI Recipe Generator!
Type a dish name like 'butter chicken', 'chocolate cake', or 'pasta'.
Type 'exit' to quit.

🍲 What recipe do you want? → biryani

🔍 Searching for recipes related to: 'biryani'...

🍽️  Recipe: Biryani

🧂 Ingredients:
 c("saffron", "milk", "hot green chili peppers", "onions", "garlic", "clove", "peppercorns", "cardamom seed", "cumin seed", "poppy seed", "mace", "cilantro", "mint leaf", "fresh lemon juice", "plain yogurt", "boneless chicken", "salt", "ghee", "onion", "tomatoes", "basmati rice", "long-grain rice", "raisins", "cashews", "eggs")

👨‍🍳 Instructions:
 c("Soak saffron in warm milk for 5 minutes and puree in blender.", "Add chiles, onions, ginger, garlic, cloves, peppercorns, cardamom seeds,